In [2]:
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, GlobalAveragePooling2D
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, TensorBoard
from keras.optimizers import Adam, SGD, Nadam, RMSprop
import cv2
import sys
import matplotlib.pyplot as plt
import os

In [3]:
# Load weights of convolution layers
from keras import backend as K
vgg_model = VGGFace(include_top=False,weights='vggface', input_shape=(200,200,3))
print('CNN Weights Loaded.')

CNN Weights Loaded.


In [4]:
#Training FC layers while freezing remaining CNN layers
nb_class = 4
hidden_dim = 512

last_layer = vgg_model.get_layer('pool5').output
x = Flatten(name='flatten')(last_layer)
x = Dense(hidden_dim, activation='relu', name='fc6')(x)
x = Dense(hidden_dim, activation='relu', name='fc7')(x)
out = Dense(nb_class, activation='softmax', name='fc8')(x)
custom_vgg_model = Model(vgg_model.input, out)

# Finetuning: only the last 3 fully connected layers are trained. The rest are made non-trainable.
layer_count = 0
for layer in custom_vgg_model.layers:
    layer_count = layer_count+1
for l in range(layer_count-3):
    custom_vgg_model.layers[l].trainable=False
    
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 200, 200, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 200, 200, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 100, 100, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 100, 100, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 100, 100, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 50, 50, 128)       0         
__________

In [5]:
# Loading train, test and validation data
train_data_dir = '/Users/seba/desktop/Datasets/Dataset/train'
validation_data_dir = '/Users/seba/desktop/Datasets/Dataset/validate'
#test_data_dir = '/Users/seba/Desktop/YaleDataset/test'
nb_train_samples = 12
nb_validation_samples = 12
nb_test_samples = 8 # per class
img_height = 200
img_width = 200 
batch_size = 4

In [6]:
MEAN_VALUE = np.array([103.939, 116.779, 123.68])   # BGR
def preprocess(img):
    # img is (channels, height, width), values are 0-255
    img = img[::-1]  # switch to BGR
    img -= MEAN_VALUE
    return img

In [7]:
# prepare data augmentation configuration
validate_datagen = ImageDataGenerator(
    preprocessing_function=preprocess,
    rescale=1./255,  ######
    featurewise_center=True,
    featurewise_std_normalization=True,
    samplewise_center=False,
    samplewise_std_normalization=False)


train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess,
    rescale=1./255,
    featurewise_center=True, 
    featurewise_std_normalization=True,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    samplewise_center=False,
    samplewise_std_normalization=False)


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validate_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 48 images belonging to 4 classes.
Found 48 images belonging to 4 classes.


In [8]:
weights_path = '/Users/seba/desktop/Weights/weights_12.h5'
#log_path = '/Users/seba/Desktop/Logs/log_12/'
#checkpointer = ModelCheckpoint(filepath=weights_path, verbose=1,save_best_only = True, monitor = 'val_loss')

In [9]:
# load best weights 
custom_vgg_model.load_weights(weights_path)

# Compile model
custom_vgg_model.compile(optimizer=SGD(lr=0.01, momentum=0.7, decay=1e-4), loss='categorical_crossentropy',metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8,patience=0, min_lr=1e-6)
#tensorboard_callback=TensorBoard(log_dir='log_path, histogram_freq=0,write_graph=True,write_images=False)


In [10]:
# Finetuning: train last 3 fully connected layers
custom_vgg_model.fit_generator( 
    train_generator,
    steps_per_epoch=nb_train_samples/batch_size,
    epochs=0,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples/batch_size)
#callbacks=[checkpointer,tensorboard_callback]

/Users/seba/anaconda/lib/python2.7/site-packages/keras/preprocessing/image.py:506: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/Users/seba/anaconda/lib/python2.7/site-packages/keras/preprocessing/image.py:514: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [ ]:
# Test network on testing data
# Testing the model 
res = dict((v,k) for k,v in train_generator.class_indices.items())
fil = open('/home/sebayoussef/Desktop/Predictions/predictions5', 'w+')

for fn in os.listdir('/Users/seba/desktop/Datasets/Dataset/test/'):
    img = imread('/Users/seba/desktop/Datasets/Dataset/test/'+fn,mode='RGB')
    img = imresize(arr,size=(200,200,3))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    # preprocessing on input data
    x = preprocess_input(x)
    i = model.predict(x)
    i = np.argmax(i,axis=1)[0]
    fil.write(fn + ',' + res[i] + '\n')

fil.close()


# evaluate the model
    cvscores1 = []
for train, test in kfold.split(X_reduced,Y):
    scores1 = model.evaluate(X_reduced[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores1[1]*100))
    cvscores1.append(scores1[1] * 100)
print("Average Accuracy: %.2f%% " % (np.mean(cvscores1)))

In [20]:
res = dict((v,k) for k,v in train_generator.class_indices.items())
print res



file=cv2.imread(filename)
file=cv2.resize(file,(224,224))
file=cv2.cvtColor(file, cv2.COLOR_BGR2RGB)
file=np.array(image).reshape((3,224,224))
files.append([file])
print(file.shape[0])
>>> x = np.random.randint(0,10,(5,5,3))
    >>> x.shape
    >>> (5, 5, 3)
    >>> x = np.expand_dims(x, axis=0)
    >>> x.shape
    >>> (1, 5, 5, 3)



SyntaxError: invalid syntax (<ipython-input-20-d574d14e8981>, line 12)

In [84]:
res = dict((v,k) for k,v in train_generator.class_indices.items())
res1 = np.array(res)
x = np.expand_dims(res1, axis=0)
x[0]

{0: 'Maryam', 1: 'Seba', 2: 'Tarek', 3: 'Youssef'}

In [149]:
import keras.utils
from keras import utils as np_utils

x=[1,1,]
x = np.asarray(x)
x = np.expand_dims(x, axis=0)
x.shape
y_test = keras.utils.to_categorical(np.random.randint(4, size=(1, 1)), num_classes=4)
y_test[0][:]=1

print y_test

[[ 1.  1.  1.  1.]]


In [154]:
label=-1
y_test = keras.utils.to_categorical(np.random.randint(4, size=(1, 1)), num_classes=4)  

avgscore=[]
for dir in os.listdir('/Users/seba/desktop/Datasets/Dataset/test/'):
    if (dir!='.DS_Store'):
        label=label+1
        y_test[0][:]=0   #initialize all labels to zero
        y_test[0][label]=1  # current label
 
        for fn in os.listdir('/Users/seba/desktop/Datasets/Dataset/test/'+dir):
            if fn.endswith(".jpg"):
                img = cv2.imread('/Users/seba/desktop/Datasets/Dataset/test/'+dir+'/'+fn,1) # read as RGB 
                img = cv2.resize(img, (200,200))
                x = np.asarray(img, dtype='float32')
                x = preprocess(x)
                # rescaling
                x= x* 1./255
                x = np.expand_dims(x, axis=0)
                print x.shape
                # preprocessing on input data
                score = custom_vgg_model.evaluate(x,y_test,verbose=0)
                #print score
                avgscores.append(score[1] * 100)

                print("%s: %.2f%%" % (custom_vgg_model.metrics_names[1], score[1]*100))
                
######## shape of label ??????


 
    
    


(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 0.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%
(1, 200, 200, 3)
acc: 100.00%


In [ ]:
# Train last FC layer or train whole network including CNN layers? 
# Validation_steps??
# Subtract mean of images?? 


#The number of samples processed for each epoch is batch_size * steps_per_epochs.
#steps_per_epoch = number of times weights are updated per epoch